In [18]:
import numpy as np
from itertools import product
from functools import partial

# Given S-box
sbox = [
    0x63, 0x7C, 0x77, 0x7B, 0xF2, 0x6B, 0x6F, 0xC5, 0x30, 0x01, 0x67, 0x2B, 0xFE, 0xD7, 0xAB, 0x76,
    0xCA, 0x82, 0xC9, 0x7D, 0xFA, 0x59, 0x47, 0xF0, 0xAD, 0xD4, 0xA2, 0xAF, 0x9C, 0xA4, 0x72, 0xC0,
    0xB7, 0xFD, 0x93, 0x26, 0x36, 0x3F, 0xF7, 0xCC, 0x34, 0xA5, 0xE5, 0xF1, 0x71, 0xD8, 0x31, 0x15,
    0x04, 0xC7, 0x23, 0xC3, 0x18, 0x96, 0x05, 0x9A, 0x07, 0x12, 0x80, 0xE2, 0xEB, 0x27, 0xB2, 0x75,
    0x09, 0x83, 0x2C, 0x1A, 0x1B, 0x6E, 0x5A, 0xA0, 0x52, 0x3B, 0xD6, 0xB3, 0x29, 0xE3, 0x2F, 0x84,
    0x53, 0xD1, 0x00, 0xED, 0x20, 0xFC, 0xB1, 0x5B, 0x6A, 0xCB, 0xBE, 0x39, 0x4A, 0x4C, 0x58, 0xCF,
    0xD0, 0xEF, 0xAA, 0xFB, 0x43, 0x4D, 0x33, 0x85, 0x45, 0xF9, 0x02, 0x7F, 0x50, 0x3C, 0x9F, 0xA8,
    0x51, 0xA3, 0x40, 0x8F, 0x92, 0x9D, 0x38, 0xF5, 0xBC, 0xB6, 0xDA, 0x21, 0x10, 0xFF, 0xF3, 0xD2,
    0xCD, 0x0C, 0x13, 0xEC, 0x5F, 0x97, 0x44, 0x17, 0xC4, 0xA7, 0x7E, 0x3D, 0x64, 0x5D, 0x19, 0x73,
    0x60, 0x81, 0x4F, 0xDC, 0x22, 0x2A, 0x90, 0x88, 0x46, 0xEE, 0xB8, 0x14, 0xDE, 0x5E, 0x0B, 0xDB,
    0xE0, 0x32, 0x3A, 0x0A, 0x49, 0x06, 0x24, 0x5C, 0xC2, 0xD3, 0xAC, 0x62, 0x91, 0x95, 0xE4, 0x79,
    0xE7, 0xC8, 0x37, 0x6D, 0x8D, 0xD5, 0x4E, 0xA9, 0x6C, 0x56, 0xF4, 0xEA, 0x65, 0x7A, 0xAE, 0x08,
    0xBA, 0x78, 0x25, 0x2E, 0x1C, 0xA6, 0xB4, 0xC6, 0xE8, 0xDD, 0x74, 0x1F, 0x4B, 0xBD, 0x8B, 0x8A,
    0x70, 0x3E, 0xB5, 0x66, 0x48, 0x03, 0xF6, 0x0E, 0x61, 0x35, 0x57, 0xB9, 0x86, 0xC1, 0x1D, 0x9E,
    0xE1, 0xF8, 0x98, 0x11, 0x69, 0xD9, 0x8E, 0x94, 0x9B, 0x1E, 0x87, 0xE9, 0xCE, 0x55, 0x28, 0xDF,
    0x8C, 0xA1, 0x89, 0x0D, 0xBF, 0xE6, 0x42, 0x68, 0x41, 0x99, 0x2D, 0x0F, 0xB0, 0x54, 0xBB, 0x16
]

# Helper function to compute the Hamming weight of a number
def hamming_weight(num):
    return bin(num).count('1')

# Compute the truth table of a Boolean function
def compute_truth_table(func, n):
    return [func(x) for x in range(2**n)]

# Generate all affine functions over GF(2^n)
def generate_affine_functions(n):
    affine_functions = []
    for a in range(2**n):  # Coefficients for linear terms
        for b in range(2):  # Constant term
            affine_functions.append(partial(lambda x, a, b: hamming_weight(x & a) % 2 ^ b, a=a, b=b))
    return affine_functions

# Non-linearity computation
def compute_non_linearity(truth_table, n):
    affine_functions = generate_affine_functions(n)
    min_distance = 2**n
    for affine in affine_functions:
        affine_table = compute_truth_table(affine, n)
        distance = sum(tt != at for tt, at in zip(truth_table, affine_table))
        min_distance = min(min_distance, distance)
    return min_distance

# Initialize parameters
n = 8  # Number of bits
sbox_bin = [format(byte, '08b') for byte in sbox]
non_linearity_table = np.zeros((n, n))  # 2D table for (j, k)

# Compute non-linearity for the entire S-box
for j, k in product(range(n), repeat=2):
    if j != k:
        # Compute the Boolean function f_jk(x) = Bit_j(S(x)) XOR Bit_k(S(x))
        def f_jk(x, j=j, k=k):
            original = int(sbox_bin[x], 2)
            bit_j = (original >> j) & 1
            bit_k = (original >> k) & 1
            return bit_j ^ bit_k

        # Compute the truth table of f_jk
        truth_table = compute_truth_table(f_jk, n)

        # Compute non-linearity of f_jk
        non_linearity_table[j][k] = compute_non_linearity(truth_table, n)

# Output the single non-linearity table
print(non_linearity_table)

[[  0. 112. 112. 112. 112. 112. 112. 112.]
 [112.   0. 112. 112. 112. 112. 112. 112.]
 [112. 112.   0. 112. 112. 112. 112. 112.]
 [112. 112. 112.   0. 112. 112. 112. 112.]
 [112. 112. 112. 112.   0. 112. 112. 112.]
 [112. 112. 112. 112. 112.   0. 112. 112.]
 [112. 112. 112. 112. 112. 112.   0. 112.]
 [112. 112. 112. 112. 112. 112. 112.   0.]]


In [26]:
# Define the S-box again for clarity in this cell
sbox = [
0x00, 0xAE, 0x95, 0x3B, 0xC4, 0x51, 0x6A, 0xA3, 0x5C, 0xCF, 0xB8, 0x30, 0x47, 0x88, 0x0A, 0x77,
0xF5, 0x3C, 0xC3, 0xD9, 0x26, 0xDA, 0x25, 0xFF, 0x69, 0x96, 0xF4, 0x0B, 0x5B, 0x09, 0xA4, 0xF6,
0xF3, 0x45, 0x0C, 0xBA, 0x2F, 0xAB, 0xD0, 0xDC, 0x54, 0x23, 0xB0, 0x57, 0x4F, 0xA8, 0xEF, 0x46,
0x10, 0x5F, 0xB9, 0xA0, 0x55, 0xAA, 0x78, 0x07, 0x87, 0x58, 0xE2, 0x42, 0x1D, 0xBD, 0xA5, 0x76,
0x5A, 0x89, 0x17, 0x70, 0x33, 0x8F, 0xCC, 0x4D, 0xB2, 0x43, 0x75, 0xB6, 0x8A, 0x49, 0x1C, 0xE3,
0x7D, 0xB1, 0x4E, 0x35, 0xED, 0xCA, 0x59, 0xE8, 0xB7, 0x48, 0xC8, 0x80, 0x37, 0x86, 0x84, 0x82,
0x7B, 0x81, 0xD3, 0x18, 0x2C, 0xE7, 0x72, 0x8D, 0xA6, 0x0D, 0x02, 0xF2, 0x14, 0x3A, 0xC5, 0xBC,
0xFC, 0x40, 0x03, 0x90, 0x1E, 0x6F, 0xE1, 0xA7, 0xFD, 0xA2, 0xEB, 0x7E, 0x62, 0x12, 0x60, 0x9F,
0x93, 0x0E, 0xF1, 0xF8, 0x74, 0x05, 0x8B, 0xFA, 0xB5, 0x4A, 0x1F, 0xE0, 0x3E, 0x68, 0x6D, 0x97,
0x92, 0xE5, 0x79, 0xAF, 0xAC, 0x50, 0xD6, 0x06, 0xF9, 0xBF, 0xC2, 0xC7, 0x3D, 0x38, 0x6C, 0xDF,
0xD2, 0x2D, 0x8E, 0x71, 0xFE, 0x01, 0xEC, 0x1A, 0x99, 0xAD, 0xD8, 0x29, 0x7C, 0x3F, 0xC0, 0x19,
0xD7, 0x28, 0xD4, 0x2B, 0x22, 0xDD, 0x04, 0x66, 0x21, 0xC1, 0x9A, 0xA9, 0xCE, 0x31, 0x32, 0xF7,
0xCD, 0x56, 0x94, 0x73, 0x83, 0x61, 0xEA, 0x15, 0x24, 0x41, 0x65, 0xBE, 0x9D, 0xA1, 0x6B, 0xBB,
0xF0, 0xE9, 0x0F, 0x8C, 0x52, 0x6E, 0xFB, 0x13, 0xB4, 0x4B, 0xDE, 0xD1, 0x2E, 0x7F, 0x08, 0x20,
0x2A, 0xD5, 0x53, 0xE6, 0x7A, 0x85, 0x1B, 0xE4, 0x34, 0xCB, 0x39, 0xC6, 0x9E, 0x91, 0x16, 0x20,
0x27, 0x5D, 0x98, 0x36, 0x44, 0x5E, 0x9C, 0x63, 0x67, 0xC9, 0x9B, 0x64, 0xEE, 0x11, 0xB3, 0x4C
]

n = 8  # Number of bits
sbox_bin = [format(byte, '08b') for byte in sbox]

# Initialize storage for results
total_nonlinearity = 0
min_nonlinearity = float('inf')
max_nonlinearity = float('-inf')
non_linearities = []

# Compute and print non-linearity for each component function
# We iterate through all possible non-zero output masks (b from 1 to 255)
for b in range(1, 2**n):
    # Define the component function f_b(x) = b . S(x) (dot product)
    # This is equivalent to XORing the bits of S(x) where the corresponding bit in b is 1
    def f_b(x, b=b):
        s_x = int(sbox_bin[x], 2)
        result = 0
        for i in range(n):
            if (b >> i) & 1:
                result ^= (s_x >> i) & 1
        return result

    # Compute the truth table of f_b
    truth_table = compute_truth_table(f_b, n)

    # Compute non-linearity of f_b
    nl_b = compute_non_linearity(truth_table, n)
    # print(f"Non-linearity for b={b}: {nl_b}")

    total_nonlinearity += nl_b
    min_nonlinearity = min(min_nonlinearity, nl_b)
    max_nonlinearity = max(max_nonlinearity, nl_b)
    non_linearities.append(nl_b)

# Final output
print(f"Total sum of non-linearities: {total_nonlinearity}")
print(f"Minimum non-linearity: {min_nonlinearity}")
print(f"Maximum non-linearity: {max_nonlinearity}")

# Calculate and print average non-linearity (with decimal)
# We divide by 2**n - 1 because we exclude the case where b=0
average_nonlinearity = total_nonlinearity / (2**n - 1)
print(f"Average non-linearity: {average_nonlinearity}")

# Print all computed non-linearities
print("\nNon-linearities for each component function (b=1 to 255):")
print(non_linearities)

Total sum of non-linearities: 26096
Minimum non-linearity: 85
Maximum non-linearity: 108
Average non-linearity: 102.33725490196079

Non-linearities for each component function (b=1 to 255):
[103, 103, 102, 102, 105, 99, 98, 105, 104, 102, 101, 103, 106, 98, 105, 105, 104, 102, 103, 101, 106, 104, 105, 100, 99, 99, 102, 102, 99, 107, 102, 97, 106, 100, 103, 103, 102, 104, 97, 104, 99, 99, 104, 100, 107, 103, 102, 102, 105, 105, 100, 100, 105, 103, 102, 97, 104, 106, 101, 101, 100, 102, 95, 103, 104, 102, 103, 101, 98, 94, 101, 104, 99, 103, 102, 108, 103, 107, 106, 98, 103, 101, 100, 104, 107, 101, 106, 105, 98, 100, 103, 105, 98, 104, 107, 106, 103, 103, 104, 106, 103, 95, 100, 103, 106, 98, 105, 97, 104, 106, 107, 103, 104, 104, 107, 99, 92, 106, 103, 98, 95, 103, 106, 104, 103, 101, 100, 105, 104, 104, 99, 105, 104, 100, 101, 98, 105, 97, 104, 106, 101, 103, 98, 106, 99, 103, 104, 96, 99, 107, 102, 105, 100, 104, 97, 105, 98, 104, 99, 102, 107, 99, 102, 104, 99, 101, 96, 85, 100, 104

In [27]:
import numpy as np

# Helper function to compute Walsh-Hadamard transform
def compute_walsh_hadamard_transform(bool_func, n):
    size = 2 ** n
    transform = [0] * size
    for x in range(size):
        for y in range(size):
            parity = bin(x & y).count('1') % 2
            transform[x] += (-1) ** (bool_func[y] ^ parity)
    return transform

# Compute non-linearity of a Boolean function
def compute_non_linearity_of_boolean_function(bool_func, n):
    walsh_transform = compute_walsh_hadamard_transform(bool_func, n)
    max_walsh_value = max(abs(val) for val in walsh_transform)
    return (2 ** (n - 1)) - (max_walsh_value // 2)

# Extract Boolean functions from S-box output bits
def extract_boolean_functions_from_sbox(s_box):
    n = int(np.log2(len(s_box)))  # input bits
    m = int(np.log2(max(s_box) + 1))  # output bits
    bool_functions = [[] for _ in range(m)]
    for x in range(len(s_box)):
        output = s_box[x]
        for i in range(m):
            bool_functions[i].append((output >> i) & 1)
    return bool_functions

# Compute non-linearities for all Boolean functions from the S-box
def compute_non_linearity_for_sbox(s_box):
    n = int(np.log2(len(s_box)))
    bool_functions = extract_boolean_functions_from_sbox(s_box)
    return [compute_non_linearity_of_boolean_function(f, n) for f in bool_functions]

# Input S-box
s_box = [
    0x63, 0x7C, 0x77, 0x7B, 0xF2, 0x6B, 0x6F, 0xC5, 0x30, 0x01, 0x67, 0x2B, 0xFE, 0xD7, 0xAB, 0x76,
    0xCA, 0x82, 0xC9, 0x7D, 0xFA, 0x59, 0x47, 0xF0, 0xAD, 0xD4, 0xA2, 0xAF, 0x9C, 0xA4, 0x72, 0xC0,
    0xB7, 0xFD, 0x93, 0x26, 0x36, 0x3F, 0xF7, 0xCC, 0x34, 0xA5, 0xE5, 0xF1, 0x71, 0xD8, 0x31, 0x15,
    0x04, 0xC7, 0x23, 0xC3, 0x18, 0x96, 0x05, 0x9A, 0x07, 0x12, 0x80, 0xE2, 0xEB, 0x27, 0xB2, 0x75,
    0x09, 0x83, 0x2C, 0x1A, 0x1B, 0x6E, 0x5A, 0xA0, 0x52, 0x3B, 0xD6, 0xB3, 0x29, 0xE3, 0x2F, 0x84,
    0x53, 0xD1, 0x00, 0xED, 0x20, 0xFC, 0xB1, 0x5B, 0x6A, 0xCB, 0xBE, 0x39, 0x4A, 0x4C, 0x58, 0xCF,
    0xD0, 0xEF, 0xAA, 0xFB, 0x43, 0x4D, 0x33, 0x85, 0x45, 0xF9, 0x02, 0x7F, 0x50, 0x3C, 0x9F, 0xA8,
    0x51, 0xA3, 0x40, 0x8F, 0x92, 0x9D, 0x38, 0xF5, 0xBC, 0xB6, 0xDA, 0x21, 0x10, 0xFF, 0xF3, 0xD2,
    0xCD, 0x0C, 0x13, 0xEC, 0x5F, 0x97, 0x44, 0x17, 0xC4, 0xA7, 0x7E, 0x3D, 0x64, 0x5D, 0x19, 0x73,
    0x60, 0x81, 0x4F, 0xDC, 0x22, 0x2A, 0x90, 0x88, 0x46, 0xEE, 0xB8, 0x14, 0xDE, 0x5E, 0x0B, 0xDB,
    0xE0, 0x32, 0x3A, 0x0A, 0x49, 0x06, 0x24, 0x5C, 0xC2, 0xD3, 0xAC, 0x62, 0x91, 0x95, 0xE4, 0x79,
    0xE7, 0xC8, 0x37, 0x6D, 0x8D, 0xD5, 0x4E, 0xA9, 0x6C, 0x56, 0xF4, 0xEA, 0x65, 0x7A, 0xAE, 0x08,
    0xBA, 0x78, 0x25, 0x2E, 0x1C, 0xA6, 0xB4, 0xC6, 0xE8, 0xDD, 0x74, 0x1F, 0x4B, 0xBD, 0x8B, 0x8A,
    0x70, 0x3E, 0xB5, 0x66, 0x48, 0x03, 0xF6, 0x0E, 0x61, 0x35, 0x57, 0xB9, 0x86, 0xC1, 0x1D, 0x9E,
    0xE1, 0xF8, 0x98, 0x11, 0x69, 0xD9, 0x8E, 0x94, 0x9B, 0x1E, 0x87, 0xE9, 0xCE, 0x55, 0x28, 0xDF,
    0x8C, 0xA1, 0x89, 0x0D, 0xBF, 0xE6, 0x42, 0x68, 0x41, 0x99, 0x2D, 0x0F, 0xB0, 0x54, 0xBB, 0x16
]

# Compute and print non-linearities
non_linearities = compute_non_linearity_for_sbox(s_box)
print("Non-linearities of all Boolean functions:", non_linearities)

# Calculate and print the average non-linearity of all Boolean functions
average_all_boolean_functions = sum(non_linearities) / len(non_linearities)
print(f"Average non-linearity of all Boolean functions: {average_all_boolean_functions}")

Non-linearities of all Boolean functions: [112, 112, 112, 112, 112, 112, 112, 112]
Average non-linearity of all Boolean functions: 112.0
